In [15]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
import os
import glob
import re

In [1]:
"""
mean boundaries computation
taking mean of N1 N2 N4
"""

'\nmean boundaries computation\ntaking mean of N1 N2 N4\n'

In [2]:
"""
Loudness Levels - from 45dB
"""

'\nLoudness Levels - from 45dB\n'

In [6]:
import numpy as np

In [4]:
"""
calculation of mean boundaries using mean boundaries for N1 N2 and N4
"""

'\ncalculation of mean boundaries using mean boundaries for N1 N2 and N4\n'

In [7]:
boundaries1 = np.array([0.63679854007417,1.24043747184364,1.90051054875325,2.19052315291591,2.42542041587323,2.62882317400665,2.76529989452493,2.88996614389811,3.01463240497312])

In [8]:
boundaries2 = np.loadtxt('C:/users/vinee/Documents/BTP/n2_avg_r1/boundaries_n2.txt')

In [9]:
boundaries4 = np.loadtxt('C:/users/vinee/Documents/BTP/n4_avg/boundaries_n4.txt')

In [12]:
mean_boundaries = (boundaries1+boundaries2+boundaries4)/3

In [13]:
mean_boundaries

array([0.64532726, 1.22833311, 1.84400641, 2.16665321, 2.40371516,
       2.60913904, 2.75278729, 2.88418764, 3.01558801])

In [14]:
import numpy as np
#%EWT functions
def ewt_vineet(f,boundaries):
    # signal spectrum
    ff = np.fft.fft(f)
    ff = abs(ff[0:int(np.ceil(ff.size / 2))])  # one-sided magnitude

    #extend the signal by mirroring to deal with boundaries
    ltemp = int(np.ceil(f.size/2)) #to behave the same as matlab's round
    fMirr =  np.append(np.flip(f[0:ltemp-1],axis = 0),f)
    fMirr = np.append(fMirr,np.flip(f[-ltemp-1:-1],axis = 0))
    ffMirr = np.fft.fft(fMirr)
    # build the corresponding filter bank
    mfb = EWT_Meyer_FilterBank(boundaries, ffMirr.size)

    # filter the signal to extract each subband
    ewt = np.zeros(mfb.shape)
    for k in range(mfb.shape[1]):
        ewt[:, k] = np.real(np.fft.ifft(np.conjugate(mfb[:, k]) * ffMirr))
    ewt = ewt[ltemp - 1:-ltemp, :]

    return ewt


def EWT_Meyer_FilterBank(boundaries, Nsig):
    """
     =========================================================================
     function mfb=EWT_Meyer_FilterBank(boundaries,Nsig)

     This function generate the filter bank (scaling function + wavelets)
     corresponding to the provided set of frequency segments

     Input parameters:
       -boundaries: vector containing the boundaries of frequency segments (0
                    and pi must NOT be in this vector)
       -Nsig: signal length

     Output:
       -mfb: cell containing each filter (in the Fourier domain), the scaling
             function comes first and then the successive wavelets

     Author: Jerome Gilles
     Institution: UCLA - Department of Mathematics
     Year: 2012
     Version: 1.0

     Python Version: Vinícius Rezende Carvalho - vrcarva@ufmg.br
     Universidade Federal de Minas Gerais - Brasil
     Núcleo de Neurociências
     =========================================================================
     """
    Npic = len(boundaries)
    # compute gamma
    gamma = 1
    for k in range(Npic - 1):
        r = (boundaries[k + 1] - boundaries[k]) / (boundaries[k + 1] + boundaries[k])
        if r < gamma:
            gamma = r
    r = (np.pi - boundaries[Npic - 1]) / (np.pi + boundaries[Npic - 1])
    if r < gamma:
        gamma = r
    gamma = (1 - 1 / Nsig) * gamma  # this ensure that gamma is chosen as strictly less than the min

    mfb = np.zeros([Nsig, Npic + 1])

    # EWT_Meyer_Scaling
    Mi = int(np.floor(Nsig / 2))
    w = np.fft.fftshift(np.linspace(0, 2 * np.pi - 2 * np.pi / Nsig, num=Nsig))
    w[0:Mi] = -2 * np.pi + w[0:Mi]
    aw = abs(w)
    yms = np.zeros(Nsig)
    an = 1. / (2 * gamma * boundaries[0])
    pbn = (1. + gamma) * boundaries[0]
    mbn = (1. - gamma) * boundaries[0]
    for k in range(Nsig):
        if aw[k] <= mbn:
            yms[k] = 1
        elif ((aw[k] >= mbn) and (aw[k] <= pbn)):
            yms[k] = np.cos(np.pi * EWT_beta(an * (aw[k] - mbn)) / 2)
    yms = np.fft.ifftshift(yms)
    mfb[:, 0] = yms

    # generate rest of the wavelets
    for k in range(Npic - 1):
        mfb[:, k + 1] = EWT_Meyer_Wavelet(boundaries[k], boundaries[k + 1], gamma, Nsig)

    mfb[:, Npic] = EWT_Meyer_Wavelet(boundaries[Npic - 1], np.pi, gamma, Nsig)

    return mfb
def EWT_beta(x):
    """
    Beta = EWT_beta(x)
    function used in the construction of Meyer's wavelet
    """
    if x<0:
        bm=0
    elif x>1:
        bm=1
    else:
        bm=(x**4)*(35.-84.*x+70.*(x**2)-20.*(x**3))
    return bm
def EWT_Meyer_Wavelet(wn,wm,gamma,Nsig):
    """
    =========================================================
    ymw=EWT_Meyer_Wavelet(wn,wm,gamma,N)
    
    Generate the 1D Meyer wavelet in the Fourier
    domain associated to scale segment [wn,wm] 
    with transition ratio gamma
    
    Input parameters:
      -wn : lower boundary
      -wm : upper boundary
      -gamma : transition ratio
      -N : number of point in the vector
    
    Output:
      -ymw: Fourier transform of the wavelet on the band [wn,wm]
    Author: Jerome Gilles
    Institution: UCLA - Department of Mathematics
    Year: 2012
    Version: 1.0
    
    Python Version: Vinícius Rezende Carvalho - vrcarva@ufmg.br
    Universidade Federal de Minas Gerais - Brasil
    Núcleo de Neurociências 
    ==========================================================            
    """
    Mi=int(np.floor(Nsig/2))
    w=np.fft.fftshift(np.linspace(0,2*np.pi - 2*np.pi/Nsig,num = Nsig))
    w[0:Mi]=-2*np.pi+w[0:Mi]
    aw=abs(w)
    ymw=np.zeros(Nsig)
    an=1./(2*gamma*wn)
    am=1./(2*gamma*wm)
    pbn=(1.+gamma)*wn
    mbn=(1.-gamma)*wn
    pbm=(1.+gamma)*wm
    mbm=(1.-gamma)*wm

    for k in range(Nsig):
        if ((aw[k]>=pbn) and (aw[k]<=mbm)):
            ymw[k]=1
        elif ((aw[k]>=mbm) and (aw[k]<=pbm)):
            ymw[k]=np.cos(np.pi*EWT_beta(am*(aw[k]-mbm))/2)
        elif ((aw[k]>=mbn) and (aw[k]<=pbn)):
            ymw[k]=np.sin(np.pi*EWT_beta(an*(aw[k]-mbn))/2)
    ymw=np.fft.ifftshift(ymw)
    return ymw

In [69]:
x_train = []
y_train = []

In [17]:
"""
Generation of dataset using N1,R1
"""

'\nGeneration of dataset using N1,R1\n'

In [19]:
os.chdir('resampled_dataset_f1_n1/resampled_dataset/')

In [20]:
my_files1 = glob.glob('*txt')

In [68]:
len(my_files1)

131000

In [34]:
idx = []
for i in range(len(my_files1)):
    string1 = my_files1[i]
    p = list(map(int, re.findall(r'\d+', string1)))
    if p[-2] == 1 and p[-3] == 1 and int(p[-1])>40:
        idx.append(i)

In [36]:
len(idx)

12

In [44]:
loudness_data_n1 = np.array([0.203489,0.461973,0.421456,0.525431,0.691075,0.733513,0.761766,1.076224,1.245589,1.340460,1.642738,1.773085])

In [45]:
len(loudness_data_n1)

12

In [46]:
loudness_data_n2 = np.array([-0.767364,-0.476870,-0.193201,0.078374,0.172603,0.419829,0.500441,0.720262,0.920215,1.080807,1.295526,1.501556])


In [48]:
len(loudness_data_n1)

12

In [56]:
loudness_data_n4 = np.array([0.502216,0.559076,0.654683,0.955243,0.968337,0.885020,0.965291,1.290606,1.397530,1.460671,1.729242,1.781090])


In [57]:
len(loudness_data_n4)

12

In [70]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files1)):
        string1 = my_files1[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files1[m])
        ewt = ewt_vineet(file,mean_boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n1)

In [71]:
len(x_train)

1000

In [74]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files1)):
        string1 = my_files1[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files1[m])
        ewt = ewt_vineet(file,mean_boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n1)

In [77]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files1)):
        string1 = my_files1[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files1[m])
        ewt = ewt_vineet(file,mean_boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n1)

In [78]:
len(x_train)

3000

In [79]:
len(y_train)

3000

In [80]:
x = np.array(x_train)

In [81]:
y = np.array(y_train)

In [82]:
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_X_cnn_n1',x)
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_Y_cnn_n1',y)

In [83]:
pwd

'C:\\Users\\vinee\\Documents\\BTP\\resampled_dataset_f1_n1\\resampled_dataset'

In [84]:
cd .. ..

C:\Users\vinee\Documents\BTP


In [92]:
os.chdir('resampled_dataset_f1_n2/')

In [93]:
my_files2 = glob.glob('*txt')

In [94]:
len(my_files2)

101000

In [ ]:
"""
Generation of dataset using N2
"""

In [96]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files2)):
        string1 = my_files2[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files2[m])
        ewt = ewt_vineet(file,mean_boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [97]:
len(x_train)

4000

In [98]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files2)):
        string1 = my_files2[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files2[m])
        ewt = ewt_vineet(file,mean_boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [99]:
len(x_train)

5000

In [100]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files2)):
        string1 = my_files2[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files2[m])
        ewt = ewt_vineet(file,mean_boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [101]:
len(x_train)

6000

In [102]:
x = np.array(x_train)

In [103]:
y = np.array(y_train)

In [104]:
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_X_cnn_n2',x)
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_Y_cnn_n2',y)

In [105]:
cd ..

C:\Users\vinee\Documents\BTP\resampled_dataset_f1_n2


In [106]:
cd ..

C:\Users\vinee\Documents\BTP


In [107]:
os.chdir('resampled_dataset_f1_n4/')

In [108]:
my_files3 = glob.glob('*txt')

In [110]:
"""
Generation of dataset using N4
"""

'\nGeneration of dataset using N4\n'

In [111]:
len(my_files3)

93000

In [112]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files3)):
        string1 = my_files3[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files3[m])
        ewt = ewt_vineet(file,mean_boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n4)

In [113]:
len(x_train)

7000

In [114]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files3)):
        string1 = my_files3[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files3[m])
        ewt = ewt_vineet(file,mean_boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n4)

In [115]:
len(x_train)

8000

In [116]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files3)):
        string1 = my_files3[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files3[m])
        ewt = ewt_vineet(file,mean_boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n4)

In [117]:
len(x_train)

9000

In [118]:
x = np.array(x_train)
y = np.array(y_train)

In [119]:
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_X_cnn',x)
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_Y_cnn',y)

In [120]:
y.shape

(9000, 12)

In [121]:
x.shape

(9000, 12, 10, 126)